### **CARTPOLE DQN KERAS**

This is a naive version of the CARTPOLE DQN algorithm <br>
Try to fill the missing parts

This is the simplest approach of DQN with a single network (Remember the standard DQN uses 2 networks and trains one, in this one we use one and train one

Fill the empty places and try to make it to work Ask for help if stocked!!!


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import random
import gymnasium as gym
import time
import numpy as np
from collections import deque
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

# Use mixed-precision training for faster computations on supported GPUs
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

import sys
sys.stderr = open('err.txt', 'w')

ModuleNotFoundError: No module named 'gymnasium'

In [2]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()
gpu_devices = [device for device in devices if device.device_type == 'GPU']
for gpu in gpu_devices:
    print('Using', gpu.physical_device_desc)

In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))
print(tf.test.is_gpu_available())

TensorFlow version: 2.10.1
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


#### **Hyperparameters**

In [3]:
# Parameters to fine tune
# Try your own parameters
# Remember epsilon and gamma are very important


MAX_EPISODES = 300
ROLLING_WINDOW = 20
MEMORY_SIZE = 2000
MAX_STEPS = 500

gamma = 0.99                        # discount rate
epsilon = 1.0                        # exploration rate
epsilon_min = 0.01
epsilon_decay = 0.99
learning_rate = 0.001
batch_size = 64
solved_threshold = 195

verb = 0                             # to see traces (verbosity)

In [4]:
# Environment setup
env = gym.make("CartPole-v1")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

#### **Neural Network**

In [10]:
###
# [CREATE YOUR NEURAL NETWORK TRY 16/32 or 24/24]
###

def build_model(state_size, action_size):
    inputs = Input(shape=(state_size,), name="state_input")
    x = Dense(16, activation='relu', name="dense_1")(inputs)
    x = Dense(32, activation='relu', name="dense_2")(x)
    outputs = Dense(action_size, activation='linear', name="output_layer")(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="Q_Network")
    model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate))
    return model

#### **Support Functions**

$Q_{\text{target}}(s_t, a_t) = r_t + \gamma \cdot \max_{a'} Q(s_{t+1}, a') \cdot (1 - \text{done})$

In [16]:
replay_buffer = deque(maxlen=MEMORY_SIZE)

def store(state, action, reward, next_state, done):             
    """
    This function appends the actual observation and reward to the Replay buffer
    """
    state = state.squeeze()
    next_state = next_state.squeeze()
    replay_buffer.append((state, action, reward, next_state, done))

def select_action_greedy(state, DQN):           
    """
    Selects the if agent takes a random action (explore) or a predicted action (exploit)
    """
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)
    act_values = DQN.predict(state, verbose=verb)
    return np.argmax(act_values[0])  # returns action selected with greedy strategy

# Sample experiences from the replay buffer
def sample_experiences(batch_size):
    """
    Samples a batch_size of experiences from the Replay buffer. 
    You MUST transform the data into numpy arrays as this accelerates the response time sensibily
    """
    indices = np.random.choice(len(replay_buffer), batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]
    states, actions, rewards, next_states, dones = zip(*batch)

    states = np.array(states)
    actions = np.array(actions)
    rewards = np.array(rewards)
    next_states = np.array(next_states)
    dones = np.array(dones)

    return (states, actions, rewards, next_states, dones)
###
# YOU MUST VECTIORIZE THE RETURN. TRANSFORM THE OUTPUTS IN np arrays otherwise it will be very slow
###

    
def experience_replay(batch_size, model, epsilon):
    """
    The critical function in the whole program
    1. gets a minibatch from replay buffer
    2. Predicts targets_qs from states (we need the full batch predicted but we'll avoid dones)
    3. predicts next_qs from next_states
    4. Bellman equation on next_qs obtains new target_qs $Q_{\text{target}}(s_t, a_t) = r_t + \gamma \cdot \max_{a'} Q(s_{t+1}, a') \cdot (1 - \text{done})$

    6. Train DQN input states, output target_qs 

    """
    if len(replay_buffer) < batch_size:
        return

    states, actions, rewards, next_states, dones = sample_experiences(batch_size)
    
    # Predict Q-values for current and next states using vectorized operations

    
#### 
#  [ Predict target_qs with model predict states, predict next_qs with model predict next states]
####
    target_qs = model.predict(states, verbose=verb) # Get current Q-values for all states
    next_qs = model.predict(next_states, verbose=verb) # Get next Q-values for next states

    # Update target Q-values using standard DQN logic   
    target_qs[np.arange(batch_size), actions] = rewards + gamma * np.max(next_qs, axis=1) * (1 - dones)
    
    # Train the model on the Q-values
    model.fit(states, target_qs, epochs=1, verbose=0)

###
#[Here you have a load and save weights. Can you make a breakpoint to restart the program after a while of training?]
###

def load(name, DQN):
    DQN.load_weights(name)

def save(name, DQN):
    DQN.save_weights(name)

DQN = build_model(state_size, action_size)
DQN.summary()

Model: "Q_Network"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ state_input (InputLayer)        │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast_4 (Cast)                   │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 690 (2.70 KB)

 Trainable params: 690 (2.70 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
rewards_per_episode= []

done = False
rolling_avg = 0
rolling_avg_rewards = []

start_time = time.time()

for e in range(MAX_EPISODES):                           # Should be While True, however we limit number of eps
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0
    
    for step in range(MAX_STEPS):
            
        action = select_action_greedy(state, DQN)
        next_state, reward, done, truncated , _ = env.step(action)
                    
        next_state = np.reshape(next_state, [1, state_size])
        store(state, action, reward, next_state, done)
        state = next_state
        total_reward = total_reward + reward
        if done:
            break
            
        if len(replay_buffer) > batch_size:
            experience_replay(batch_size, DQN, epsilon)
            
    epsilon = max(epsilon_min, epsilon * epsilon_decay)            # decay epsilon
    
    rewards_per_episode.append(total_reward)
    rolling_avg = np.mean(rewards_per_episode[-ROLLING_WINDOW:])   # append rewards
    rolling_avg_rewards.append(rolling_avg)
    print(f"Episode: {e+1:3}/{MAX_EPISODES}, Reward: {total_reward:+7.2f}, "
          f"Epsilon: {epsilon:.2f}, Rolling Avg: {rolling_avg:6.2f}, Steps: {step:3} Terminated: {done} ")

            
    # Check if environment is solved
    if rolling_avg >= solved_threshold:
        print(f"Environment solved in {e+1} episodes!")
#            model.save("lunarlander_ddqn_model1.keras")
        break

end_time = time.time()
testing_duration = (end_time - start_time) / 60  # Convert to minutes
print(f"Testing completed in {testing_duration:.2f} minutes")

Episode:   1/300, Reward:  +15.00, Epsilon: 0.90, Rolling Avg:  15.00, Steps:  14 Terminated: True 
Episode:   2/300, Reward:  +16.00, Epsilon: 0.89, Rolling Avg:  15.50, Steps:  15 Terminated: True 
Episode:   3/300, Reward:  +45.00, Epsilon: 0.88, Rolling Avg:  25.33, Steps:  44 Terminated: True 
Episode:   4/300, Reward:   +9.00, Epsilon: 0.87, Rolling Avg:  21.25, Steps:   8 Terminated: True 
Episode:   5/300, Reward:  +28.00, Epsilon: 0.86, Rolling Avg:  22.60, Steps:  27 Terminated: True 
Episode:   6/300, Reward:  +11.00, Epsilon: 0.85, Rolling Avg:  20.67, Steps:  10 Terminated: True 
Episode:   7/300, Reward:  +28.00, Epsilon: 0.84, Rolling Avg:  21.71, Steps:  27 Terminated: True 
Episode:   8/300, Reward:  +12.00, Epsilon: 0.83, Rolling Avg:  20.50, Steps:  11 Terminated: True 
Episode:   9/300, Reward:  +12.00, Epsilon: 0.83, Rolling Avg:  19.56, Steps:  11 Terminated: True 
Episode:  10/300, Reward:  +13.00, Epsilon: 0.82, Rolling Avg:  18.90, Steps:  12 Terminated: True 


KeyboardInterrupt: 

#### **Learning Plot and Episode Rewards**

In [ ]:
# Plot rewards with rolling average

plt.figure(figsize=(10, 6))
plt.plot(rewards_per_episode, label='Rewards', color='blue')
plt.plot(rolling_avg_rewards, label='Rolling Avg (Last 20 Episodes)', color='orange')
plt.axhline(y=solved_threshold, color='red', linestyle='--', label='Solved Threshold')
plt.title('DQN Training Performance CARTPOLE')
plt.xlabel('Episodes')
plt.ylabel('Rewards')
plt.legend()
plt.grid()
plt.show()

#### **Simulation - Testing 10 episodes with the DQN Network**

In [ ]:
# Testing for 10 episodes
start_time = time.time()

for e_test in range(10):  # Run 10 test episodes
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0
    
    steps = 0
    while True:
        # Use the trained model for testing
        action_vals = DQN.predict(state, verbose=0)  # Predict action values
        action = np.argmax(action_vals[0])  # Choose the action with the highest Q-value

        next_state, reward, done, truncated, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        total_reward += reward
        state = next_state
        steps = steps + 1
        if done or truncated:
            print(f"Test Episode: {e_test + 1:2}/10, Reward: {total_reward:.2f}, Steps: {steps:3}")
            break

end_time = time.time()
testing_duration = (end_time - start_time) / 60  # Convert to minutes
print(f"Testing completed in {testing_duration:.2f} minutes")

#### **Rendering 1 episode**

In [ ]:
# Test the trained agent with video rendering
# This code is useful if you are using colab otherwise use render_mode='human'
env = gym.make(("CartPole-v1"), render_mode='rgb_array')  # Enable RGB rendering
frames = []  # Store frames for visualization

# Render a single test episode
state, _ = env.reset()
state = np.reshape(state, [1, state_size])
tot_rewards = 0

while True:
    # Use the trained model for action
    action_vals = DQN.predict(state, verbose=0)  # Predict action values
    action = np.argmax(action_vals[0])           # Choose the action with the highest Q-value

    next_state, reward, done, truncated, _ = env.step(action)
    frames.append(env.render())                  # Save frame for rendering later
    next_state = np.reshape(next_state, [1, state_size])
    tot_rewards += reward
    state = next_state

    if done or truncated:
        print(f"Rendered Test Episode Reward: {tot_rewards:.2f}")
        break

env.close()

# Save the rendered episode as a GIF
def save_frames_as_gif(frames, path='./', filename='CARTPOLE_DQN.gif'):
    images = [Image.fromarray(frame) for frame in frames]
    gif_path = os.path.join(path, filename)
    images[0].save(gif_path, save_all=True, append_images=images[1:], duration=50, loop=0)
    print(f"Saved GIF to: {gif_path}")

save_frames_as_gif(frames, filename='CARTPOLE_DQN.gif')


In [ ]:
import session_info
session_info.show(html=False)